In [ ]:
# # SETUP ENVIRONMENT
import os
import sys

# # Set Java (SỬA PATH NÀY!)
os.environ['JAVA_HOME'] = 'C:\\Java\\jdk-1.8'

# # QUAN TRỌNG: Bypass Hadoop requirement
os.environ['HADOOP_HOME'] = os.environ.get('JAVA_HOME')
os.environ['PATH'] = f"{os.environ['JAVA_HOME']}\\bin;{os.environ.get('PATH', '')}"

print(f" JAVA_HOME: {os.environ['JAVA_HOME']}")

In [ ]:
# INSTALL PACKAGES
!pip install pyspark findspark matplotlib seaborn -q

In [ ]:
# IMPORT LIBRARIES
import findspark
findspark.init()

from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Set style cho plots
plt.style.use('default')
sns.set_palette("husl")


In [ ]:
# INITIALIZE SPARK SESSION
import tempfile

spark = SparkSession.builder \
    .appName("YouTubeTimeAnalysis") \
    .master("local[1]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.warehouse.dir", tempfile.gettempdir()) \
    .config("spark.ui.enabled", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print(f"Spark {spark.version} started")

In [ ]:
# LOAD PREPROCESSED DATA
df = spark.read.csv("./data/preprocessed_data.csv", header=True, inferSchema=True)

print(f" Loaded {df.count():,} rows with {len(df.columns)} columns")
print("\nSAMPLE DATA")
df.show(5)

In [ ]:
# DATA PREPARATION

# Convert timestamps
df = df.withColumn('trending_date', to_timestamp('trending_date'))
df = df.withColumn('publishedAt', to_timestamp('publishedAt'))

# Extract time features
df = df.withColumn('publish_hour', hour('publishedAt'))
df = df.withColumn('publish_day', date_format('publishedAt', 'E'))  # Mon, Tue, Wed
df = df.withColumn('publish_date', to_date('publishedAt'))
df = df.withColumn('trending_day', date_format('trending_date', 'E'))
df = df.withColumn('trending_month', month('trending_date'))
df = df.withColumn('trending_year', year('trending_date'))

print(" Time features extracted")
df.select('video_id', 'publishedAt', 'publish_hour', 'publish_day').show(5)

In [ ]:
# Thêm đoạn này vào cuối cell 'prepare_data'
df = df.withColumn("category_name", 
    when(col("categoryId") == "1", "Film & Animation")
    .when(col("categoryId") == "2", "Autos & Vehicles")
    .when(col("categoryId") == "10", "Music")
    .when(col("categoryId") == "15", "Pets & Animals")
    .when(col("categoryId") == "17", "Sports")
    .when(col("categoryId") == "19", "Travel & Events")
    .when(col("categoryId") == "20", "Gaming")
    .when(col("categoryId") == "22", "People & Blogs")
    .when(col("categoryId") == "23", "Comedy")
    .when(col("categoryId") == "24", "Entertainment")
    .when(col("categoryId") == "25", "News & Politics")
    .when(col("categoryId") == "26", "Howto & Style")
    .when(col("categoryId") == "27", "Education")
    .when(col("categoryId") == "28", "Science & Technology")
    .otherwise("Unknown")
)

In [ ]:
# BƯỚC CHUẨN BỊ BỔ SUNG: TẠO DF VỚI METRICS
# Cần DF này cho phân tích engagement theo giờ

# print(" CHUẨN BỊ BỔ SUNG: Tạo DF metrics độc nhất")

# Convert metrics
df_with_metrics = df.withColumn("view_count_num", col("view_count").cast("long")) \
    .withColumn("likes_num", col("likes").cast("long")) \
    .withColumn("dislikes_num", col("dislikes").cast("long")) \
    .withColumn("comment_count_num", col("comment_count").cast("long"))

# Lấy view/like/comment cao nhất cho mỗi video
window_spec_metrics = Window.partitionBy("video_id").orderBy(desc("view_count_num"))
df_unique_with_metrics = df_with_metrics.withColumn("rank", row_number().over(window_spec_metrics)) \
    .filter(col("rank") == 1) \
    .drop("rank")

print(f"Đã tạo df_unique_with_metrics với {df_unique_with_metrics.count()} video độc nhất")
# df_unique_with_metrics giờ đã có publish_hour, view_count_num, likes_num
df_unique_with_metrics.select("video_id", "publish_hour", "view_count_num", "likes_num").show(5)

In [ ]:
# PHÂN TÍCH 1: HOURLY UPLOAD PATTERNS

print("HOURLY UPLOAD PATTERNS")

# Group by publish hour
hourly_analysis = df.groupBy('publish_hour') \
    .count() \
    .orderBy('publish_hour')

print("\n Videos by Upload Hour:")
hourly_analysis.show(24)

# Convert to Pandas for plotting
hourly_analysis_pd = hourly_analysis.toPandas()

# Plot
plt.figure(figsize=(16, 6))
plt.bar(hourly_analysis_pd['publish_hour'], hourly_analysis_pd['count'], color='steelblue', alpha=0.8)
plt.xlabel('Hour of Day (0-23)', fontsize=12)
plt.ylabel('Number of Videos', fontsize=12)
plt.title('Video Upload Distribution by Hour', fontsize=14, fontweight='bold')
plt.xticks(range(0, 24))
plt.grid(axis='y', alpha=0.3)

# Add value labels
for i, row in hourly_analysis_pd.iterrows():
    plt.text(row['publish_hour'], row['count'] + 100, f"{row['count']:,}", 
             ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

# Find peak hours
peak_hour = hourly_analysis_pd.loc[hourly_analysis_pd['count'].idxmax()]
low_hour = hourly_analysis_pd.loc[hourly_analysis_pd['count'].idxmin()]

print(f"\nPeak upload hour: {int(peak_hour['publish_hour'])}:00 ({peak_hour['count']:,} videos)")
print(f"Lowest upload hour: {int(low_hour['publish_hour'])}:00 ({low_hour['count']:,} videos)")

In [ ]:
# PHÂN TÍCH 2: DAY OF WEEK PATTERNS

print(" DAY OF WEEK PATTERNS")

# Group by day of week
dow_analysis = df.groupBy('publish_day') \
    .count() \
    .orderBy('count', ascending=False)

print("\n Videos by Day of Week:")
dow_analysis.show()

# Convert to Pandas
dow_analysis_pd = dow_analysis.toPandas()

# Reorder days properly
day_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
dow_analysis_pd['publish_day'] = pd.Categorical(
    dow_analysis_pd['publish_day'], 
    categories=day_order, 
    ordered=True
)
dow_analysis_pd = dow_analysis_pd.sort_values('publish_day')

# Plot
plt.figure(figsize=(14, 6))
bars = plt.bar(dow_analysis_pd['publish_day'], dow_analysis_pd['count'], 
               color='coral', alpha=0.8)
plt.xlabel('Day of Week', fontsize=12)
plt.ylabel('Number of Videos', fontsize=12)
plt.title('Video Upload Distribution by Day of Week', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 100,
             f'{int(height):,}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

# Find peak day
peak_day = dow_analysis_pd.loc[dow_analysis_pd['count'].idxmax()]
print(f"\nPeak upload day: {peak_day['publish_day']} ({peak_day['count']:,} videos)")

In [ ]:
# PHÂN TÍCH 3: TRENDING DAY PATTERNS

print("TRENDING DAY PATTERNS")

# Group by trending day
trending_dow = df.groupBy('trending_day') \
    .count() \
    .orderBy('count', ascending=False)

print("\nVideos by Trending Day:")
trending_dow.show()

trending_dow_pd = trending_dow.toPandas()
trending_dow_pd['trending_day'] = pd.Categorical(
    trending_dow_pd['trending_day'],
    categories=day_order,
    ordered=True
)
trending_dow_pd = trending_dow_pd.sort_values('trending_day')

# Plot
plt.figure(figsize=(14, 6))
plt.bar(trending_dow_pd['trending_day'], trending_dow_pd['count'], 
        color='green', alpha=0.7)
plt.xlabel('Day of Week', fontsize=12)
plt.ylabel('Number of Videos', fontsize=12)
plt.title('Trending Videos by Day of Week', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# PHÂN TÍCH 4: TIME TO TRENDING

print("TIME TO TRENDING ANALYSIS")

# Calculate days from publish to trending
df_time = df.withColumn(
    'days_to_trend',
    datediff(col('trending_date'), col('publishedAt'))
)

# Get first trending date for each video
window_spec = Window.partitionBy('video_id').orderBy('trending_date')
df_first_trend = df_time.withColumn('rank', row_number().over(window_spec)) \
    .filter(col('rank') == 1) \
    .drop('rank')

# Statistics
stats = df_first_trend.select(
    avg('days_to_trend').alias('avg_days'),
    expr('percentile_approx(days_to_trend, 0.5)').alias('median_days'),
    min('days_to_trend').alias('min_days'),
    max('days_to_trend').alias('max_days')
).collect()[0]

print(f"\nTime to Trending Statistics:")
print(f"   Average: {stats['avg_days']:.1f} days")
print(f"   Median: {stats['median_days']} days")
print(f"   Min: {stats['min_days']} days")
print(f"   Max: {stats['max_days']} days")

# Distribution
days_dist = df_first_trend.groupBy('days_to_trend') \
    .count() \
    .orderBy('days_to_trend') \
    .limit(30)

days_dist_pd = days_dist.toPandas()

# Plot
plt.figure(figsize=(14, 6))
plt.bar(days_dist_pd['days_to_trend'], days_dist_pd['count'], color='purple', alpha=0.7)
plt.xlabel('Days from Upload to Trending', fontsize=12)
plt.ylabel('Number of Videos', fontsize=12)
plt.title('Distribution: Days from Upload to First Trending', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Same-day trending
same_day = df_first_trend.filter(col('days_to_trend') == 0).count()
total = df_first_trend.count()
pct = (same_day / total * 100)

print(f"\nSame-day trending: {same_day:,} videos ({pct:.1f}%)")

In [ ]:
# PHÂN TÍCH 5: HOUR x DAY HEATMAP

print("UPLOAD PATTERNS: HOUR x DAY HEATMAP")

# Create pivot table
hour_day = df.groupBy('publish_hour', 'publish_day') \
    .count() \
    .toPandas()

pivot = hour_day.pivot(index='publish_hour', 
                        columns='publish_day', 
                        values='count').fillna(0)

# Reorder columns
pivot = pivot[day_order]

# Plot heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(pivot, annot=True, fmt='.0f', cmap='YlOrRd', 
            cbar_kws={'label': 'Number of Videos'})
plt.xlabel('Day of Week', fontsize=12)
plt.ylabel('Hour of Day', fontsize=12)
plt.title('Upload Activity Heatmap: Hour x Day', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Find peak hour-day combination
max_idx = pivot.stack().idxmax()
max_val = pivot.stack().max()
print(f"\nPeak activity: {max_idx[1]} at {int(max_idx[0])}:00 ({max_val:.0f} videos)")

In [ ]:
# PHÂN TÍCH BỔ SUNG 1: TIME-TO-TREND THEO THỂ LOẠI
# Mục đích: Xem thể loại nào "bùng nổ" nhanh nhất.

print("PHÂN TÍCH BỔ SUNG: TIME-TO-TREND THEO THỂ LOẠI")

# df_first_trend đã được tạo ở Phân tích 4
# Và df đã được map category_name (từ bước 1)

time_to_trend_category = df_first_trend.groupBy("category_name") \
    .agg(
        avg("days_to_trend").alias("avg_days_to_trend"),
        expr("percentile_approx(days_to_trend, 0.5)").alias("median_days_to_trend")
    ) \
    .orderBy("avg_days_to_trend") # Sắp xếp theo ngày trung bình TĂNG DẦN

print("Thời gian trung bình để lên Top Trending theo Thể loại (Nhanh nhất -> Chậm nhất)")
time_to_trend_category.show()

# Plotting
time_trend_cat_pd = time_to_trend_category.toPandas()
plt.figure(figsize=(15, 7))
bars = plt.bar(time_trend_cat_pd['category_name'], time_trend_cat_pd['avg_days_to_trend'])
plt.title('Thời gian trung bình lên Top Trending (theo Thể loại)', fontsize=16)
plt.xlabel('Thể loại')
plt.ylabel('Số ngày trung bình')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("Insight: 'News & Politics' và 'Music' (cho MV mới) thường có")
print("thời gian lên top rất ngắn (trung bình 1-2 ngày).")

In [ ]:
# PHÂN TÍCH BỔ SUNG 2: TƯƠNG TÁC THEO GIỜ ĐĂNG
# Mục đích: Giờ nào đăng video thì hiệu quả (nhiều tương tác) nhất?

print("PHÂN TÍCH BỔ SUNG: TƯƠNG TÁC THEO GIỜ ĐĂNG")

# Sử dụng df_unique_with_metrics từ cell chuẩn bị bổ sung
engagement_by_hour = df_unique_with_metrics.groupBy("publish_hour") \
    .agg(
        avg("view_count_num").alias("avg_views"),
        avg("likes_num").alias("avg_likes"),
        avg("comment_count_num").alias("avg_comments")
    ) \
    .orderBy("publish_hour")

print("Tương tác trung bình theo giờ đăng video")
engagement_by_hour.show(24)

# Plotting
engagement_hour_pd = engagement_by_hour.toPandas()

fig, axes = plt.subplots(3, 1, figsize=(16, 15), sharex=True)
fig.suptitle('Tương tác trung bình theo Giờ đăng video', fontsize=16, y=1.02)

sns.lineplot(data=engagement_hour_pd, x='publish_hour', y='avg_views', ax=axes[0], marker='o')
axes[0].set_title('Lượt xem trung bình')
axes[0].set_ylabel('Avg Views')
axes[0].grid(axis='both', alpha=0.3)

sns.lineplot(data=engagement_hour_pd, x='publish_hour', y='avg_likes', ax=axes[1], marker='o', color='green')
axes[1].set_title('Likes trung bình')
axes[1].set_ylabel('Avg Likes')
axes[1].grid(axis='both', alpha=0.3)

sns.lineplot(data=engagement_hour_pd, x='publish_hour', y='avg_comments', ax=axes[2], marker='o', color='red')
axes[2].set_title('Comments trung bình')
axes[2].set_ylabel('Avg Comments')
axes[2].grid(axis='both', alpha=0.3)

plt.xlabel('Giờ đăng (0-23)')
plt.xticks(range(0, 24))
plt.tight_layout()
plt.show()

print(" Insight: So sánh biểu đồ này với biểu đồ 'SỐ LƯỢNG video' theo giờ.")
print(" Có thể giờ đăng nhiều nhất (cao điểm) không phải là giờ có tương tác trung bình cao nhất.")

In [ ]:
# PHÂN TÍCH BỔ SUNG 3: "TUỔI THỌ" TRENDING
# Mục đích: Video trụ lại top trending trong bao lâu?

print("PHÂN TÍCH BỔ SUNG: TUỔI THỌ TRENDING")

# Sử dụng df (đã map category_name)
# Tính số ngày trending cho mỗi video
trending_lifespan_video = df.groupBy("video_id", "category_name") \
    .agg(
        (datediff(max("trending_date"), min("trending_date")) + 1).alias("days_on_trending")
    )

# Tính trung bình theo thể loại
trending_lifespan_category = trending_lifespan_video.groupBy("category_name") \
    .agg(
        avg("days_on_trending").alias("avg_trending_lifespan"),
        expr("percentile_approx(days_on_trending, 0.5)").alias("median_trending_lifespan")
    ) \
    .orderBy(desc("avg_trending_lifespan"))

print("Tuổi thọ trung bình trên Top Trending (theo Thể loại)")
trending_lifespan_category.show()

# Plotting
lifespan_pd = trending_lifespan_category.toPandas()
plt.figure(figsize=(15, 7))
bars = plt.bar(lifespan_pd['category_name'], lifespan_pd['avg_trending_lifespan'])
plt.title('Tuổi thọ trung bình trên Top Trending (theo Thể loại)', fontsize=16)
plt.xlabel('Thể loại')
plt.ylabel('Số ngày trung bình')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("Insight: Các video 'Music' có thể bùng nổ nhanh nhưng cũng 'nguội' nhanh,")
print("trong khi 'Gaming' hoặc 'Howto' có thể trụ lại lâu hơn.")

In [ ]:
# EXPORT TIME BASELINE FOR REAL-TIME MONITORING

from datetime import datetime
import json
import os

print("EXPORTING TIME BASELINE METRICS")

# Thu thập time-based metrics
baseline_time_metrics = {
    'timestamp': datetime.now().isoformat(),
    'analysis_type': 'time_analysis',
    
    # 1. Hourly patterns
    'hourly_distribution': {
        'video_count_by_hour': hourly_analysis_pd.set_index('publish_hour')['count'].to_dict(),
        'peak_hours': hourly_analysis_pd.nlargest(3, 'count')['publish_hour'].astype(int).tolist(),
        'low_hours': hourly_analysis_pd.nsmallest(3, 'count')['publish_hour'].astype(int).tolist()
    },
    
    # 2. Day of week patterns
    'daily_distribution': {
        'video_count_by_day': dow_analysis_pd.set_index('publish_day')['count'].to_dict(),
        'peak_days': dow_analysis_pd.nlargest(3, 'count')['publish_day'].tolist(),
        'low_days': dow_analysis_pd.nsmallest(3, 'count')['publish_day'].tolist()
    },
    
    # 3. Trending day patterns
    'trending_distribution': {
        'video_count_by_trending_day': trending_dow_pd.set_index('trending_day')['count'].to_dict()
    },
    
    # 4. Time to trending
    'time_to_trend': {
        'avg_days': float(stats['avg_days']),
        'median_days': int(stats['median_days']),
        'min_days': int(stats['min_days']),
        'max_days': int(stats['max_days']),
        'same_day_percentage': float(pct)
    },
    
    # 5. Statistical summary
    'statistics': {
        'most_active_hour': int(hourly_analysis_pd.loc[hourly_analysis_pd['count'].idxmax(), 'publish_hour']),
        'least_active_hour': int(hourly_analysis_pd.loc[hourly_analysis_pd['count'].idxmin(), 'publish_hour']),
        'most_active_day': str(dow_analysis_pd.loc[dow_analysis_pd['count'].idxmax(), 'publish_day']),
        'avg_videos_per_hour': float(hourly_analysis_pd['count'].mean()),
        'avg_videos_per_day': float(dow_analysis_pd['count'].mean())
    }
}

# Lưu vào file JSON
os.makedirs('./data/baselines', exist_ok=True)
baseline_path = './data/baselines/time_baseline.json'

with open(baseline_path, 'w', encoding='utf-8') as f:
    json.dump(baseline_time_metrics, f, indent=2, ensure_ascii=False)

print(f"\nSaved time baseline metrics to: {baseline_path}")

# In ra summary
print(f"\nTIME BASELINE SUMMARY:")
print(f"   • Peak upload hour: {baseline_time_metrics['statistics']['most_active_hour']}:00")
print(f"   • Peak upload day: {baseline_time_metrics['statistics']['most_active_day']}")
print(f"   • Avg videos/hour: {baseline_time_metrics['statistics']['avg_videos_per_hour']:.0f}")
print(f"   • Avg days to trending: {baseline_time_metrics['time_to_trend']['avg_days']:.1f}")
print(f"   • Same-day trending: {baseline_time_metrics['time_to_trend']['same_day_percentage']:.1f}%")

print(f"\nTOP 3 PEAK HOURS:")
for hour in baseline_time_metrics['hourly_distribution']['peak_hours']:
    count = baseline_time_metrics['hourly_distribution']['video_count_by_hour'][hour]
    print(f"   • {hour}:00 - {count:,} videos")

print(f"\nTOP 3 PEAK DAYS:")
for day in baseline_time_metrics['daily_distribution']['peak_days']:
    count = baseline_time_metrics['daily_distribution']['video_count_by_day'][day]
    print(f"   • {day} - {count:,} videos")

print(f"\nUSAGE:")
print(f"   This baseline will be used to detect:")
print(f"   • Unusual upload time patterns")
print(f"   • Day-of-week trending shifts")
print(f"   • Abnormal time-to-trending")

print("Time analysis baseline export completed!")

In [ ]:
# Stop Spark
spark.stop()